In [43]:
from PIL import Image
from keras.models import load_model
from keras_facenet import FaceNet
import numpy as np
from numpy import asarray
from numpy import expand_dims
import pandas as pd
import datetime
import pickle
import cv2

In [44]:
HaarCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
MyFaceNet = FaceNet()

In [45]:
with open("data.pkl", "rb") as myFile:
    database = pickle.load(myFile)

In [46]:
# Load or create an Excel file
excel_file_path = 'face_data.xlsx'
try:
    df = pd.read_excel(excel_file_path)
except FileNotFoundError:
    # If the file doesn't exist, create a new DataFrame with columns
    df = pd.DataFrame(columns=['Name', 'Time', 'Date'])
    df.to_excel(excel_file_path, index=False)  # Save the DataFrame to the Excel file
    df = pd.read_excel(excel_file_path)  # Reload the DataFrame after saving

cap = cv2.VideoCapture(0)
cv2.namedWindow('res', cv2.WINDOW_NORMAL)

# Display window to ensure it's not minimized
_, gbr1 = cap.read()
cv2.imshow('res', gbr1)
cv2.waitKey(1)

while True:
    _, gbr1 = cap.read()

    croppedFace = HaarCascade.detectMultiScale(gbr1, 1.1, 4)

    if len(croppedFace) > 0:
        x1, y1, width, height = croppedFace[0]
    else:
        x1, y1, width, height = 1, 1, 10, 10

    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    gbr = cv2.cvtColor(gbr1, cv2.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)
    gbr_array = asarray(gbr)

    face = gbr_array[y1:y2, x1:x2]
    face = Image.fromarray(face)
    face = face.resize((160, 160))
    face = asarray(face)

    face = expand_dims(face, axis=0)
    signature = MyFaceNet.embeddings(face)

    min_dist = 100
    identity = ' '

    for key, value in database.items():
        dist = np.linalg.norm(value - signature)
        if dist < min_dist:
            min_dist = dist
            identity = key

    # Get current time and date
    current_time = datetime.datetime.now().strftime("%H:%M:%S")
    current_date = datetime.datetime.now().strftime("%Y-%m-%d")

    # Check if the DataFrame is empty or if the person's name already exists for the current date
    if df.empty or not df[(df['Name'] == identity) & (df['Date'] == current_date)].empty:
        # Check for duplicates within the DataFrame for the same date and name
        duplicate_index = df[(df['Name'] == identity) & (df['Date'] == current_date)].index
        if not duplicate_index.empty:
            # Remove duplicate entries for the same date and name
            df = df.drop(duplicate_index)
            
        # Create a new DataFrame for the current entry
        new_entry = pd.DataFrame({'Name': [identity], 'Time': [current_time], 'Date': [current_date]})

        # Concatenate the new entry with the existing DataFrame
        df = pd.concat([df, new_entry], ignore_index=True)

        # Update Excel file
        df.to_excel(excel_file_path, index=False)

    cv2.putText(gbr1, identity, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
    cv2.rectangle(gbr1, (x1, y1), (x2, y2), (0, 255, 0), 2)

    cv2.imshow('res', gbr1)

    k = cv2.waitKey(5) & 0xFF
    if k == 27 or cv2.getWindowProperty('res', cv2.WND_PROP_VISIBLE) < 1:
        break

cv2.destroyAllWindows()
cap.release()

1/1 [==============================] - 0s 73ms/step
